In [1]:
#colab
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/aml_final/aml_final/
! git pull
! pip install setfit
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive', force_remount=True)\n%cd /content/drive/MyDrive/aml_final/aml_final/\n! git pull\n! pip install setfit\n"

In [2]:
from datasets import load_dataset, Dataset
from setfit import SetFitModel, Trainer, TrainingArguments
import torch, gc

from data.dataset_config import DatasetConfig
from train.active_learning import ActiveTrainer, create_random_subset
from train.active_learning_config import ActiveLearningConfig
from train.reporter import Reporter
from train.metrics import camprehesive_metrics

In [3]:
samples_per_cycle = 12 # 12 * 4 = 8 * 6, comparable to setfit paper

In [4]:
dataset = load_dataset("dair-ai/emotion")
dataset_config = DatasetConfig(text_column="text", num_classes=6)
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

final_reporter = Reporter("final_balancing.csv", label_column=dataset_config.label_column)

In [5]:
slow_train_args = TrainingArguments(num_iterations=20, num_epochs=(1, 16))
fast_train_args = TrainingArguments(num_iterations=10, num_epochs=(1, 8))


def run_train(args, initial_train_subset, active_learning_config, model_init, **kwargs):
    trainer = ActiveTrainer(
        model_init=model_init, 
        full_train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        initial_train_subset=initial_train_subset,
        train_args=args,
        active_learning_config=active_learning_config, 
        dataset_config=dataset_config,
        metric=camprehesive_metrics,
        run_id=kwargs["run_id"],
        final_model_train_args=slow_train_args
    )
    t = trainer.train()
    final_reporter.report(
        trainer=t, 
        dataset=trainer.train_subset, 
        active_learning_config=active_learning_config, 
        dataset_name="emotion", **kwargs #kwars, so you can put anything here
        )

In [6]:
run_id = 0
for dataset_id in range(3):
    initial_subset = create_random_subset(dataset=train_dataset, num_samples=samples_per_cycle)
    for model_name in ["sentence-transformers/all-mpnet-base-v2", "WhereIsAI/UAE-Large-V1"]:
        model_init = lambda: SetFitModel.from_pretrained(model_name, use_differentiable_head=True, head_params={"out_features": dataset_config.num_classes}).to(device)
        for speed in ["fast", "slow"]:
            if "fast":
                train_args = fast_train_args
            else:
                train_args = slow_train_args
            for setting in range(5):
                strategy = "max_entropy"
                balancing = None
                if setting == 0:
                    strategy = "random"
                if setting == 2:
                    balancing = 0.1
                if setting == 3:
                    balancing = 0.25
                if setting == 4:
                    balancing = 0.5
                active_config = ActiveLearningConfig(samples_per_cycle=samples_per_cycle, model_name=model_name, unlabeled_samples=600, active_sampling_strategy=strategy, balancing_factor=balancing)
                run_train(train_args, initial_subset, active_config, model_init, run_id=run_id, dataset_id=dataset_id, setting=setting)
                gc.collect()
                run_id+=1

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

c:\Users\Colin\Documents\aml\aml_final\.venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Colin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

data_config.json:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]